Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# Set Cover problem

See: https://en.wikipedia.org/wiki/Set_cover_problem

In [421]:
import numpy as np
from matplotlib import pyplot as plt
from dataclasses import dataclass
import functools

from icecream import ic

## Reproducible Initialization

If you want to get reproducible results, use `rng` (and restart the kernel); for non-reproducible ones, use `np.random`.

In [422]:
UNIVERSE_SIZE = 1_000
NUM_SETS = 1000
DENSITY = 0.2

rng = np.random.Generator(np.random.PCG64([UNIVERSE_SIZE, NUM_SETS, int(10_000 * DENSITY)]))

In [423]:
# DON'T EDIT THESE LINES!

SETS = np.random.random((NUM_SETS, UNIVERSE_SIZE)) < DENSITY
for s in range(UNIVERSE_SIZE):
    if not np.any(SETS[:, s]):
        SETS[np.random.randint(NUM_SETS), s] = True
COSTS = np.pow(SETS.sum(axis=1), 1.1)

## Helper Functions

In [424]:
@dataclass
class Individual:
    genome: np.ndarray = None
    fitness: float = None

In [425]:
def counter(fn):
    """Simple decorator for counting number of calls"""
    @functools.wraps(fn)
    def helper(*args, **kargs):
        helper.calls += 1
        return fn(*args, **kargs)

    helper.calls = 0
    return helper


def valid(solution):
    """Checks wether solution is valid (ie. covers all universe)"""
    phenotype = np.logical_or.reduce(SETS[solution])
    return np.all(phenotype)

@counter
def cost(solution):
    """Returns the cost of a solution (to be minimized)"""
    return COSTS[solution].sum()


def num_covered(solution):
    """Checks wether solution is valid (ie. covers all universe)"""
    return np.sum(np.logical_or.reduce(SETS[solution]))


def fitness(solution):
    return (int(num_covered(solution)), float(-cost(solution)))


def parent_selection(population):
    candidates = sorted(np.random.choice(population, 2), key=lambda e: e.fitness, reverse=True)
    return candidates[0]


def xover(p1: Individual, p2: Individual):
    m = np.random.rand(NUM_SETS) < .5
    genome = p1.genome.copy()
    genome[m] = p2.genome[m]
    return Individual(genome)


def mutation(p: Individual):
    genome = p.genome.copy()
    mutation_probability = 0
    while mutation_probability < .1:
        index = np.random.randint(NUM_SETS)
        genome[index] = not genome[index]
        mutation_probability = np.random.rand()
    return Individual(genome)


## Have Fun!

In [426]:
# A dumb solution of "all" sets
solution = np.full(NUM_SETS, True)
ic(valid(solution), cost(solution))
None

ic| valid(solution): np.True_
    cost(solution): np.float64(340128.5958894346)


In [427]:
# A random solution with random 50% of the sets
solution = rng.random(NUM_SETS) < .5
ic(valid(solution), cost(solution))
None

ic| valid(solution): np.True_
    cost(solution): np.float64(177221.20222166908)


# Simple GA

In [428]:
POPULATION_SIZE = 10

population = [Individual(np.random.rand(NUM_SETS) < .5) for _ in range(POPULATION_SIZE)]

for i in population:
    i.fitness = fitness(i.genome)

OFFSPRING_SIZE = 4

In [429]:
MAX_GENERATIONS = 10_000

for g in range(MAX_GENERATIONS):
    offspring = list()
    for _ in range(OFFSPRING_SIZE):
        # hyper-model
        if np.random.random() < .3:
            i = parent_selection(population)
            o = mutation(i)
        else:
            i1 = parent_selection(population)
            i2 = parent_selection(population)
            o = xover(i1, i2)
        offspring.append(o)

    for i in offspring:
        i.fitness = fitness(i.genome)

    population.extend(offspring)
    population.sort(key=lambda i: i.fitness, reverse=True)
    population = population[:POPULATION_SIZE]
ic(population[0].fitness, cost.calls)
None


ic| population[0].fitness: (1000, -6956.647349943672)
    cost.calls: 40012
